![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [115]:
# Importing and reading

import pandas as pd
import numpy as np
from collections import defaultdict

df = pd.read_csv("bank_marketing.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [116]:
# Preprocessing functions

def replace_dots(df, col):
    return df[col].str.replace(".","_")

def replace_bool(df, col):
    value_mapping = {
        'yes': 1,
        'succes':1,
        'no': 0,
        'failure':0,
        'nonexistent':0,
        'unknown': 0
    }
    df[col] = df[col].map(value_mapping)
    return df[col].astype(bool)

In [117]:
# Client
client_dict = {
    "client_id":int,
    "age":int,
    "job":object,
    "marital":object,
    "education":object,
    "credit_default":bool,
    "mortgage":bool
}

client_cols = list(client_dict.keys())
client_df = df[client_cols]

client_df["job"] = replace_dots(client_df,"job")
client_df["education"] = replace_dots(client_df,"education")
client_df.loc[client_df['education'] == 'unknown', 'education'] = np.NaN

client_df["credit_default"] = replace_bool(client_df,"credit_default")
client_df["mortgage"] = replace_bool(client_df,"mortgage")
client_df.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


In [118]:
# Campaign
campaign_dict = {
    "client_id": int,
    "number_contacts": int,
    "contact_duration": int,
    "previous_campaign_contacts": int,
    "previous_outcome": bool,
    "campaign_outcome": bool,
    "month": object,
    "day": int
}

campaign_cols = list(campaign_dict.keys())
campaign_df = df[campaign_cols]

campaign_df["previous_outcome"] = replace_bool(campaign_df,"previous_outcome")
campaign_df["campaign_outcome"] = replace_bool(campaign_df,"campaign_outcome")

campaign_df["year"] = 2022
campaign_df['month'] = pd.to_datetime(campaign_df['month'], format='%b').dt.strftime('%m')
campaign_df["last_contact_date"] = pd.to_datetime(campaign_df[["year","month","day"]]).dt.strftime('%Y-%m-%d')
campaign_df = campaign_df.drop(["year","month","day"], axis=1)
campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


In [119]:
# Economics

economics_dict = {
    "client_id": int,
    "cons_price_idx": float,
    "euribor_three_months": float
}

economics_cols = list(economics_dict.keys())
economics_df = df[economics_cols]
economics_df = economics_df.astype(economics_dict)
economics_df.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [120]:
# Saving everything

client_df.to_csv("client.csv",index=False)
campaign_df.to_csv("campaign.csv", index=False)
economics_df.to_csv("economics.csv", index=False)